# Daily data download and first processing

This worksheet is part of a project to analyse and visualise data on the COVID-19 outbreak and will be run once a day.<br>
Though this can only be done locally on my machine, the data will be saved in ../data/ in this repository.

The data used is the ECDC's daily data sheet (https://ourworldindata.org/coronavirus-source-data), containing the amount of new cases and deaths due to the COVID-19 outbreak per country per day.<br>
This will be split into three data sets: 

- Daily cases per country 
- Daily deaths per country 
- Populations of countries in 2018.

## Downloading

First we download the ECDC data sheet.

In [1]:
import urllib.request               # Calls url to downlload daily file
import datetime                     # Provides current date

try:
    today = datetime.date.today() - datetime.timedelta(days=1)
    url = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-' + str(today) + '.xlsx'

    # Second argument in urlretrieve is the destination and filename for saving
    _ = urllib.request.urlretrieve(url, '../data/ECDC_data.xlsx')
    
except: 
    print("No new data.")

Next we open the downloaded .xlsx file as a pandas data frame.<br>
Pandas is a powerful python library to create and manipulate data frames that is used throughout this project.

In [2]:
import pandas as pd                   # Tool to manipulate datasets

df = pd.read_excel('../data/ECDC_data.xlsx')
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-04-14,14,4,2020,58,3,Afghanistan,AF,AFG,37172386.0
1,2020-04-13,13,4,2020,52,0,Afghanistan,AF,AFG,37172386.0
2,2020-04-12,12,4,2020,34,3,Afghanistan,AF,AFG,37172386.0
3,2020-04-11,11,4,2020,37,0,Afghanistan,AF,AFG,37172386.0
4,2020-04-10,10,4,2020,61,1,Afghanistan,AF,AFG,37172386.0
...,...,...,...,...,...,...,...,...,...,...
10737,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
10738,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
10739,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
10740,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


## Preperations

We will clean out the data since we have quite some values that we do not need.

Considering the 'DateRep' column contains a datetime we will delete the unneeded time-attributes, as well as the GEOId's.<br>
This can be changed at a later time in case we want to combine other datasets containing country data.

In [3]:
df = df.drop(columns=['day', 'month', 'year', 'geoId', 'countryterritoryCode'])
display(df)

,dateRep,cases,deaths,countriesAndTerritories,popData2018
0,2020-04-14,58,3,Afghanistan,37172386.0
1,2020-04-13,52,0,Afghanistan,37172386.0
2,2020-04-12,34,3,Afghanistan,37172386.0
3,2020-04-11,37,0,Afghanistan,37172386.0
4,2020-04-10,61,1,Afghanistan,37172386.0
...,...,...,...,...,...
10737,2020-03-25,0,0,Zimbabwe,14439018.0
10738,2020-03-24,0,1,Zimbabwe,14439018.0
10739,2020-03-23,0,0,Zimbabwe,14439018.0
10740,2020-03-22,1,0,Zimbabwe,14439018.0


We will create two lists to work with. One containing every available date and another containing every available country. Both will be ordered so we can fill in all the datasets in the same alphabetical/ascending order.<br>
We will also exchange the _'s in the country name for a whitespace.

In [4]:
dates_all = []
countries = []

for date in df['dateRep']:
    if date not in dates_all:
        dates_all.append(date)

for country in df['countriesAndTerritories']:
    if country not in countries:
        countries.append(country)
        
dates_all.sort()
countries.sort()

countries_all = [country.replace('_', ' ') for country in countries]

We will also create lists separating the available countries per continent.<br>
These lists will be verified and manually updated every day.

In [5]:
africa = ['Sao Tome and Principe', 'Zimbabwe', 'South Sudan', 'Malawi', 'Niger', 'Nigeria', 'Zambia','United Republic of Tanzania','Uganda','Tunisia','Togo','Sudan','South Africa','Somalia','Sierra Leone','Seychelles', 'Senegal','Rwanda', 'Namibia','Mozambique','Morocco','Mauritius','Mauritania','Mali','Madagascar','Libya','Liberia', 'Kenya','Guinea Bissau','Guinea', 'Ghana', 'Gambia', 'Gabon','Ethiopia', 'Eswatini','Eritrea','Equatorial Guinea','Egypt', 'Djibouti', 'Democratic Republic of the Congo','Cote dIvoire', 'Congo', 'Chad', 'Central African Republic', 'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde']
asia = ['Yemen', 'Cases on an international conveyance Japan', 'Vietnam','Uzbekistan', 'United Arab Emirates','Turkey','Timor Leste','Thailand','Taiwan','Syria','Sri Lanka','South Korea','Singapore','Saudi Arabia','Qatar', 'Philippines','Palestine','Pakistan','Oman', 'Nepal','Myanmar', 'Mongolia','Maldives','Malaysia','Lebanon','Laos','Kyrgyzstan', 'Kuwait','Jordan', 'Japan', 'Israel','Iraq','Iran','Indonesia','India','China', 'Afghanistan', 'Armenia', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei Darussalam', 'Cambodia']
europe = ['Czechia', 'United Kingdom','Ukraine','Switzerland','Sweden','Spain','Slovenia','Slovakia','Serbia','San Marino','Romania', 'Russia','Portugal','Poland','Norway', 'North Macedonia','Netherlands','Monaco','Montenegro','Moldova','Malta','Luxembourg', 'Lithuania','Liechtenstein','Latvia','Kosovo','Kazakhstan', 'Jersey','Italy', 'Isle of Man','Ireland','Azerbaijan','Georgia','Iceland', 'Hungary', 'Holy See','Guernsey', 'Greece', 'Gibraltar', 'Germany', 'France', 'Finland', 'Faroe Islands', 'Estonia','Denmark', 'Cyprus', 'Croatia', 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria']
north_america = ['Belize', 'United States of America','United States Virgin Islands','Turks and Caicos islands','Sint Maarten','Saint Vincent and the Grenadines','Saint Lucia','Saint Kitts and Nevis','Puerto Rico', 'Panama','Nicaragua', 'Montserrat','Mexico', 'Jamaica', 'Honduras', 'Haiti', 'Guatemala', 'Grenada', 'Greenland','El Salvador','Dominican Republic', 'Dominica', 'Cuba', 'Cayman Islands', 'Anguilla', 'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Bermuda', 'British Virgin Islands', 'Canada']
oceania = ['Papua New Guinea','Northern Mariana Islands','New Zealand','New Caledonia','Guam','French Polynesia', 'Fiji','Australia']
south_america = ['Falkland Islands (Malvinas)', 'Bonaire, Saint Eustatius and Saba', 'Venezuela','Uruguay','Trinidad and Tobago','Suriname','Peru','Paraguay', 'Guyana', 'Ecuador', 'CuraÃ§ao', 'Costa Rica', 'Colombia', 'Chile', 'Argentina', 'Aruba', 'Bolivia', 'Brazil']

africa.sort()
asia.sort()
europe.sort()
north_america.sort()
south_america.sort()
oceania.sort()

continents = {'Africa': africa, 'Asia': asia, 'Europe': europe, 'North America': north_america, 'Oceania': oceania, 'South America': south_america}

missing = []
for country in countries_all:
    if not any([country in continents[entry] for entry in continents]):
        missing.append(country)
            
to_much = []
for entry in continents:
    for country in continents[entry]:
        if not country in countries_all:
            to_much.append(country)
            
if missing != [] or to_much != []:
    print(f'The follwoing countries are missing: {missing}')
    print(f'The follwoing countries are to much: {to_much}')

## Data extraction

We will create  dictionaries containing all the column names as keys and the column values as content. This will allow us to, in a structured way, fill the columns. <br>
These dictionaries will then be used to create the data frames.

We will use the dates as an index, filling each column with data from one country and naming each column the updated country names.

In [6]:
dict_cases = {'Date': dates_all}
dict_deaths = {'Date': dates_all}

# here we need a numerical for loop, instead of loping through the countries list
# since we need to place the countries_final name in the dictionary instead of the countries one
for i in range(len(countries)):
    cases = []
    deaths = []

    country_df = df.loc[df['countriesAndTerritories']==countries[i]]
    collected_dates = country_df['dateRep'].tolist()

    for date in dates_all:
        if date in collected_dates:
            entry = country_df.loc[country_df['dateRep']==date]
            cases.append(entry['cases'].values[0])
            deaths.append(entry['deaths'].values[0])

        else:
            cases.append(0)
            deaths.append(0)

    dict_cases[countries_all[i]] = cases
    dict_deaths[countries_all[i]] = deaths

df_cases_daily = pd.DataFrame(dict_cases).set_index('Date')
df_cases_daily = df_cases_daily.fillna(0)

df_deaths_daily = pd.DataFrame(dict_deaths).set_index('Date')
df_deaths_daily = df_deaths_daily.fillna(0)

We will also place the global and continental data in the data sets.<br>
We do this now since it is easier to get these sums with a data frame compared to a dictionairy.

In [7]:
df_cases_daily['Global'] = df_cases_daily.sum(axis=1).values
df_deaths_daily['Global']  = df_deaths_daily.sum(axis=1).values

for entry in continents:
    df_cases_daily[entry] = df_cases_daily[continents[entry]].sum(axis=1).values
    df_deaths_daily[entry]  = df_deaths_daily[continents[entry]].sum(axis=1).values

For ease of use later on, we will place these as the first 7 columns.<br>
for this we will create a mask for the column order that we can apply to the data frames later.

In [8]:
cols = df_cases_daily.columns.tolist()
cols = cols[-7:] + cols[:-7]

df_cases_daily = df_cases_daily[cols]
df_deaths_daily = df_deaths_daily[cols]

With have our completed data sets.<br>
Now we will just show a part of it to verify, assert that the global and continental data is correct, and finally, if the assertion succeeds, save the data set in the correct directory.

In [9]:
display(df_cases_daily)
assert((df_cases_daily.iloc[:,:1].values - df_cases_daily.iloc[:,1:7].values.sum(axis=1)).sum() == 0)

df_cases_daily.to_csv('../data/cases_daily.csv')

,Global,Africa,Asia,Europe,North America,Oceania,South America,Afghanistan,Albania,Algeria,...,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,27,0,27,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,17,0,17,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-10,87039,834,10024,35577,36351,126,4127,61,9,94,...,0,5,33901,17,27,4,4,1,0,0
2020-04-11,89349,672,10932,34161,37934,109,5541,37,7,95,...,7,0,35527,21,42,4,2,0,1,0
2020-04-12,81707,714,12966,34438,30496,68,3025,34,17,64,...,0,1,28391,7,172,0,1,0,0,3


wWe will do the same thing for the daily deaths.

In [10]:
display(df_deaths_daily)
assert((df_deaths_daily.iloc[:,:1].values - df_deaths_daily.iloc[:,1:7].values.sum(axis=1)).sum() == 0)

df_deaths_daily.to_csv('../data/deaths_daily.csv')

,Global,Africa,Asia,Europe,North America,Oceania,South America,Afghanistan,Albania,Algeria,...,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-10,7223,57,348,4588,1998,3,229,1,1,30,...,0,0,1873,0,0,1,0,0,0,1
2020-04-11,7049,61,327,4251,2212,4,194,0,0,21,...,2,0,2087,0,0,0,0,0,1,0
2020-04-12,6104,52,385,3552,1974,4,137,3,0,19,...,0,0,1831,0,1,0,0,0,0,0


We still need to extract the information on the populations.

We will again make a dictionairy, since the data frame structure will be different. This time with a single row containing the popultions.

In [11]:
dict_populations = {'Ind': 'Populations'}

for i in range(len(countries)):
    country_pop = df.loc[df['countriesAndTerritories']==countries[i]]['popData2018'].values[-1]
    dict_populations[countries_all[i]] = [country_pop]

df_populations = pd.DataFrame(dict_populations).set_index('Ind')
df_populations = df_populations.fillna(0)

Again we add the global and continental information.

In [12]:
df_populations['Global'] = df_populations.sum(axis=1).values
countries_all.append('Global')

for entry in continents:
    country_mask = [country in continents[entry] for country in countries_all]
    df_populations[entry] = df_populations.loc['Populations', country_mask].sum()
    countries_all.append(entry)

df_populations = df_populations[cols]

We finish by rearranging the columns, asserting the correct global and continental data and saveing the data set.

In [13]:
display(df_populations)
assert((df_populations.iloc[:,:1].values - df_populations.iloc[:,1:7].values.sum(axis=1)).sum() == 0)

df_populations.to_csv('../data/populations.csv')

,Global,Africa,Asia,Europe,North America,Oceania,South America,Afghanistan,Albania,Algeria,...,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
Ind,,,,,,,,,,,,,,,,,,,,,
Populations,7.538067e+09,1.268028e+09,4.444493e+09,779726977.0,575613071.0,40152057.0,430053988.0,37172386.0,2866376.0,42228429.0,...,56318348.0,106977.0,327167434.0,3449299.0,32955400.0,28870195.0,95540395.0,28498687.0,17351822.0,14439018.0
